In [ ]:
!pip install datasets
!pip install transformers
import xml.etree.ElementTree as ET
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00


In [ ]:
# 1. 解析 XML 檔案
# 假設 XML 檔案結構如下，每筆記錄用 <record> 包含四個子標籤：
# <用戶提出的問題>、<對應的節能建議>、<本節能建議能節省的電力或具體成效等量化的數據說明>、<資料來源>
#xml_file = "/content/drive/MyDrive/論文題目/論文相關資料/總整理.xlsx"  # 請將此處路徑替換為您的 XML 檔案路徑

# prompt: 讀取雲端硬碟中的csv檔案

from google.colab import drive
drive.mount('/content/drive')

# Assuming your CSV file is in your Google Drive, replace with the actual path
csv_file_path = "總整理CSV.csv"

with open(csv_file_path, "r", encoding="big5") as f:
    content = f.read()
    print(content)


df = pd.read_csv(csv_file_path, encoding="big5")
print(df.head())



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
用戶提出的問題,對應的節能建議,本節能建議能節省的電力或具體成效等量化的數據說明,資料來源
如何利用排氣冷能回收降低冷卻成本？,利用排氣冷能回收，將室內低溫排氣用來預冷進入空調箱。,? 此措施有助於降低冰水主機的耗電，但具體數值依系統條件而異。,空調系統節能技術.pdf
如何利用風機變頻技術節省電力？,採用風機變頻控制，根據室內溫度調整風機轉速以改變供風量。,? 根據負荷變化，風機在50%風量下輸入功率可降至約13%。,空調系統節能技術.pdf
如何利用室內正壓調整降低能耗？,調整室內正壓、減少不必要的外氣引入，從而降低冷卻和加熱負荷。,? 可降低加熱與冷卻能耗，具體效益視現場條件而定。,空調系統節能技術.pdf
如何採用熱回收外氣空調箱達到節能效果？,利用冰水盤管預冷外氣，再用熱水盤管回收外氣熱能，降低再加熱需求。,? 雙重節能效應可減少冷能與加熱盤管能耗，具體節省比例未明確。,空調系統節能技術.pdf
如何降低潔淨室供風溫度以節省能源？,降低潔淨室供風溫度，從而減少加熱盤管所需提供的熱能。,? 能夠減少加熱耗能，節省效果依運行狀況而定。,空調系統節能技術.pdf
如何通過冰水泵變頻控制來降低系統能耗？,採用冰水泵變頻設計，根據最末端負載信號調整泵浦轉速與流量。,? 當流量降至80%時，輸入功率約為51%；降至50%時僅約13%。,空調系統節能技術.pdf
如何避免設備過大設計導致的能耗浪費？,透過節能診斷，適當汰換或調整冷卻水泵與冰水泵，避免安全係數過大。,? 可顯著降低運轉電費，節能效果依系統設計而定。,空調系統節能技術.pdf
如何透過最佳化主機與水泵運轉降低耗電？,分析不同負載下的運轉數據，找到主機與水泵的最佳運行點。,? 在70%-85%負荷下，離心式主機效率最佳，節能效果明顯。,空調系統節能技術.pdf
如何採用熱回收冰水主機降低能源消耗？,使用熱回收冰水主機，將部分散熱量回收供其他用途，如供溫水負載。,? 此措施可同時降低冷卻塔容量及耗電，節能效益顯著。,空調系統節能技術.pdf
如何調高冰水供水

In [ ]:
from huggingface_hub import login

hf_token = "" # @param{type:“string”} 需替換
login(token=hf_token, add_to_git_credential=True)

!huggingface-cli whoami

Hsuan0929


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# 假設 CSV 的欄位名稱分別為：用戶提出的問題, 對應的節能建議, 本節能建議能節省的電力或具體成效等量化的數據說明, 資料來源
# 若欄位名稱有空格或其他特殊符號，可依實際情況調整
# 將資料轉換成 Dataset 格式
dataset = Dataset.from_pandas(df)

# 2. 建構對話訓練資料，目標回答格式為：
# "對應的節能建議：{對應的節能建議內容}\n🔴本節能建議能節省的電力或具體成效等量化的數據說明：{量化數據內容}\n資料來源：{資料來源內容}"
def preprocess_function(examples):
    inputs = []
    targets = []
    for q, s, quant, src in zip(
            examples["用戶提出的問題"],
            examples["對應的節能建議"],
            examples["本節能建議能節省的電力或具體成效等量化的數據說明"],
            examples["資料來源"]):

        # 去除各欄位前可能存在的 "? " 前綴 (可依需求處理)
        s_clean = s.lstrip("? ").strip()
        quant_clean = quant.lstrip("? ").strip()
        src_clean = src.strip()

        # 構建輸入 prompt 與目標回答
        input_text = f"用戶提出的問題：{q}\n請給出節電建議："
        target_text = (f"{s_clean}\n"
                       f"🔴{quant_clean}\n"
                       f"資料來源：{src_clean}")
        inputs.append(input_text)
        targets.append(target_text)
    return {"input_text": inputs, "target_text": targets}

processed_dataset = dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/107 [00:00<?, ? examples/s]

In [ ]:
# 3. 載入 Llama 3.2 模型與對應的 Tokenizer
# 請根據 Hugging Face 上實際的 Llama 3.2 模型名稱替換 "huggingface/llama-3.2-base"
model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:


# 4. 定義 tokenize 函數，將 prompt 與回答合併為模型輸入（使用換行符號分隔）
def tokenize_function(examples):
    # 將每個樣本的 prompt 與回答合併
    combined_texts = [inp + "\n" + tgt for inp, tgt in zip(examples["input_text"], examples["target_text"])]
    tokenized = tokenizer(combined_texts, truncation=True, max_length=512)
    return tokenized

tokenized_dataset = processed_dataset.map(tokenize_function, batched=True, remove_columns=processed_dataset.column_names)

# 5. 設定 DataCollator 以及訓練參數
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./llama_energy_saving_model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    evaluation_strategy="no",
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    push_to_hub=True,  # 啟用上傳至 Hugging Face Hub
    hub_model_id="YOUR_HF_USERNAME/llama3.2-energy-saving",  # 將 YOUR_HF_USERNAME 替換成 Hugging Face 用戶名稱
    hub_strategy="every_save"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# 6. 開始訓練並上傳模型至 Hugging Face Hub
trainer.train()
trainer.push_to_hub()

print("訓練完成，模型已上傳至 Hugging Face Hub。")